In [1]:
from importlib import reload
import flymovie as fm
from flymovie.simnuc import Sim
import cnn_models.evaluate_models as ev
import matplotlib.pyplot as plt
import scipy.ndimage as ndi
import numpy as np
import os
import random
import pickle
from importlib import reload
from flymovie.general_functions import stack_normalize_minmax
%load_ext autoreload
%autoreload 2

In [2]:
kernel = fm.load_pickle('/Users/michaelstadler/Bioinformatics/Projects/rpb1/PSFs/psf_20220210_21x25x25pixels_100x50x50voxel.pkl')
gfp_intensity=100

In [3]:
def mkdirs(dir_):
    if not os.path.isdir(dir_):
        os.mkdir(dir_)
        os.mkdir(os.path.join(dir_, 'left'))
        os.mkdir(os.path.join(dir_, 'right'))

## Isolating cluster number

In [69]:
outfolder = '/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/siamese_eval_datasets/iso_nclusters'
mkdirs(outfolder)
mask = Sim.make_spherical_mask(100,100,100,40)
sim = Sim(mask, res_z=85, res_ij=85)
eroded_coords = sim.get_eroded_coordinates(10)
eroded_coords = list(zip(eroded_coords[0], eroded_coords[1], eroded_coords[2]))

for n_clusters in range(0,150,5):
    for rep in ['left', 'right']:
        sim = Sim(mask, res_z=85, res_ij=85)
        sim.add_kernel(kernel, res_z=100, res_ij=50)
        # Add free population.
        sim.add_n_objects(10_000, gfp_intensity, fluors_per_object=1, size=1, mode='nuc')
        # Add HLB.
        sim.add_sphere(eroded_coords[np.random.randint(len(eroded_coords))], gfp_intensity, 400, 5)
        sim.add_sphere(eroded_coords[np.random.randint(len(eroded_coords))], gfp_intensity, 400, 5)
        # Add clusters.
        sim.add_n_objects(n_clusters, gfp_intensity, fluors_per_object=13, 
                size=2)
        # Add noise and convolve.
        sim.add_noise('poisson')
        sim.convolve()
        sim.resize((250,85,85), order=1)
        sim.im = stack_normalize_minmax(sim.im) * 1000
        filename = '0_0_' + str(n_clusters) + '.pkl'
        sim.add_noise('gaussian', sigma=27)
        fm.save_pickle(sim.im, os.path.join(outfolder, rep, filename))




In [92]:
im1a = fm.load_pickle('/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/siamese_eval_datasets/iso_nclusters/left/0_0_5.pkl')
im1b = fm.load_pickle('/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/siamese_eval_datasets/iso_nclusters/right/0_0_5.pkl')
im2a = fm.load_pickle('/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/siamese_eval_datasets/iso_nclusters/left/0_0_50.pkl')
im2b = fm.load_pickle('/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/siamese_eval_datasets/iso_nclusters/right/0_0_50.pkl')
fm.viewer([im1a, im1b, im2a, im2b], 6)

interactive(children=(Dropdown(description='Color', index=1, options=('gray', 'cividis', 'gray_r', 'Greens', '…

## Isolating cluster fluor number

In [71]:
outfolder = '/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/siamese_eval_datasets/iso_mols_per_cluster'
mkdirs(outfolder)
mask = Sim.make_spherical_mask(100,100,100,40)
sim = Sim(mask, res_z=85, res_ij=85)
eroded_coords = sim.get_eroded_coordinates(10)
eroded_coords = list(zip(eroded_coords[0], eroded_coords[1], eroded_coords[2]))

for fluors_per_cluster in range(3,37,3):
    for rep in ['left', 'right']:
        sim = Sim(mask, res_z=85, res_ij=85)
        sim.add_kernel(kernel, res_z=100, res_ij=50)
        # Add free population.
        sim.add_n_objects(10_000, gfp_intensity, fluors_per_object=1, size=1, mode='nuc')
        # Add HLB.
        sim.add_sphere(eroded_coords[np.random.randint(len(eroded_coords))], gfp_intensity, 400, 5)
        sim.add_sphere(eroded_coords[np.random.randint(len(eroded_coords))], gfp_intensity, 400, 5)
        # Add clusters.
        sim.add_n_objects(60, gfp_intensity, fluors_per_object=fluors_per_cluster, 
                size=2)
        # Add noise and convolve.
        sim.add_noise('poisson')
        sim.convolve()
        sim.resize((250,85,85), order=1)
        sim.im = stack_normalize_minmax(sim.im) * 1000
        filename = '0_0_' + str(fluors_per_cluster) + '.pkl'
        sim.add_noise('gaussian', sigma=27)
        fm.save_pickle(sim.im, os.path.join(outfolder, rep, filename))

In [74]:
im = fm.load_pickle('/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/siamese_eval_datasets/iso_mols_per_cluster/left/0_0_3.pkl')
fm.viewer(im, 6)

interactive(children=(Dropdown(description='Color', index=1, options=('gray', 'cividis', 'gray_r', 'Greens', '…

## Isolating nfree

In [75]:
outfolder = '/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/siamese_eval_datasets/iso_nfree'
mkdirs(outfolder)
mask = Sim.make_spherical_mask(100,100,100,40)
sim = Sim(mask, res_z=85, res_ij=85)
eroded_coords = sim.get_eroded_coordinates(10)
eroded_coords = list(zip(eroded_coords[0], eroded_coords[1], eroded_coords[2]))

for nfree in range(0, 40_000, 2500):
    for rep in ['left', 'right']:
        sim = Sim(mask, res_z=85, res_ij=85)
        sim.add_kernel(kernel, res_z=100, res_ij=50)
        # Add free population.
        sim.add_n_objects(nfree, gfp_intensity, fluors_per_object=1, size=1, mode='nuc')
        # Add HLB.
        sim.add_sphere(eroded_coords[np.random.randint(len(eroded_coords))], gfp_intensity, 400, 5)
        sim.add_sphere(eroded_coords[np.random.randint(len(eroded_coords))], gfp_intensity, 400, 5)
        # Add clusters.
        sim.add_n_objects(60, gfp_intensity, fluors_per_object=15, 
                size=2)
        # Add noise and convolve.
        sim.add_noise('poisson')
        sim.convolve()
        sim.resize((250,85,85), order=1)
        sim.im = stack_normalize_minmax(sim.im) * 1000
        filename = '0_0_' + str(nfree) + '.pkl'
        sim.add_noise('gaussian', sigma=27)
        fm.save_pickle(sim.im, os.path.join(outfolder, rep, filename))

In [78]:
im = fm.load_pickle('/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/siamese_eval_datasets/iso_nfree/left/0_0_20000.pkl')
fm.viewer(im, 6)

interactive(children=(Dropdown(description='Color', index=1, options=('gray', 'cividis', 'gray_r', 'Greens', '…

## Isolating fluors_per_HLB

In [81]:
outfolder = '/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/siamese_eval_datasets/iso_HLB_fluors'
mkdirs(outfolder)
mask = Sim.make_spherical_mask(100,100,100,40)
sim = Sim(mask, res_z=85, res_ij=85)
eroded_coords = sim.get_eroded_coordinates(10)
eroded_coords = list(zip(eroded_coords[0], eroded_coords[1], eroded_coords[2]))

for fluors_per_hlb in range(100, 801, 50):
    for rep in ['left', 'right']:
        sim = Sim(mask, res_z=85, res_ij=85)
        sim.add_kernel(kernel, res_z=100, res_ij=50)
        # Add free population.
        sim.add_n_objects(10_000, gfp_intensity, fluors_per_object=1, size=1, mode='nuc')
        # Add HLB.
        sim.add_sphere(eroded_coords[np.random.randint(len(eroded_coords))], gfp_intensity, fluors_per_hlb, 5)
        sim.add_sphere(eroded_coords[np.random.randint(len(eroded_coords))], gfp_intensity, fluors_per_hlb, 5)
        # Add clusters.
        sim.add_n_objects(60, gfp_intensity, fluors_per_object=15, 
                size=2)
        # Add noise and convolve.
        sim.add_noise('poisson')
        sim.convolve()
        sim.resize((250,85,85), order=1)
        sim.im = stack_normalize_minmax(sim.im) * 1000
        filename = '0_0_' + str(fluors_per_hlb) + '.pkl'
        sim.add_noise('gaussian', sigma=27)
        fm.save_pickle(sim.im, os.path.join(outfolder, rep, filename))

In [80]:
im = fm.load_pickle('/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/siamese_eval_datasets/iso_HLB_fluors/left/0_0_100.pkl')
fm.viewer(im, 6)

interactive(children=(Dropdown(description='Color', index=1, options=('gray', 'cividis', 'gray_r', 'Greens', '…

## Rotating images

In [110]:
infolder = '/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/siamese_eval_datasets/rpb1_narrow_vCPKRofP/left'
outfolder = '/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/siamese_eval_datasets/rotations'
mkdirs(outfolder)
mask = Sim.make_spherical_mask(100,100,100,40)
mask = ndi.zoom(mask, (85/250,1,1), order=0)
mask = ndi.morphology.binary_dilation(mask, np.ones((2,5,5)))
files = os.listdir(infolder)
for f in files:
    if f[0] == '.':
        continue
    im_orig = fm.load_pickle(os.path.join(infolder, f))
    im_rot1 = ndi.rotate(im_orig, np.random.RandomState().uniform(45, 180), axes=(1,2), reshape=False)
    fm.save_pickle(np.where(mask, im_rot1, im_orig), os.path.join(outfolder, 'left', f))
    im_rot2 = ndi.rotate(im_orig, np.random.RandomState().uniform(-180, -45), axes=(1,2), reshape=False, order=5)
    fm.save_pickle(np.where(mask, im_rot2, im_orig), os.path.join(outfolder, 'right', f))

In [111]:
im1 = fm.load_pickle('/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/siamese_eval_datasets/rotations/left/aBX_14645_10.35_406_33_2_0.55_15_1.75_34.77_rep0.pkl')
im2 = fm.load_pickle('/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/siamese_eval_datasets/rotations/right/aBX_14645_10.35_406_33_2_0.55_15_1.75_34.77_rep0.pkl')
fm.viewer([im1.max(axis=0), im2.max(axis=0)], 6)

interactive(children=(Dropdown(description='Color', index=1, options=('gray', 'cividis', 'gray_r', 'Greens', '…

In [109]:
mask = Sim.make_spherical_mask(100,100,100,40)
mask = ndi.zoom(mask, (85/250,1,1), order=0)
mask = ndi.morphology.binary_dilation(mask, np.ones((2,5,5)))
fm.viewer(np.where(mask, im1, 0), 4)

interactive(children=(Dropdown(description='Color', index=1, options=('gray', 'cividis', 'gray_r', 'Greens', '…

## Flipping images

In [5]:
infolder = '/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/siamese_eval_datasets/rpb1_narrow_vCPKRofP/left'
outfolder = '/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/siamese_eval_datasets/flips'
mkdirs(outfolder)
mask = Sim.make_spherical_mask(100,100,100,40)
mask = ndi.zoom(mask, (85/250,1,1), order=0)
mask = ndi.morphology.binary_dilation(mask, np.ones((2,5,5)))
files = os.listdir(infolder)
for f in files:
    if f[0] == '.':
        continue
    im_orig = fm.load_pickle(os.path.join(infolder, f))
    fm.save_pickle(im_orig, os.path.join(outfolder, 'left', f))
    im_flip = np.flip(im_orig, axis=1)
    fm.save_pickle(im_flip, os.path.join(outfolder, 'right', f))

In [7]:
ev.visualize_simfolder(outfolder, figsize=6)

interactive(children=(Dropdown(description='Color', index=9, options=('inferno', 'viridis', 'prism', 'Greens',…

## Rotating90

In [8]:
infolder = '/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/siamese_eval_datasets/rpb1_narrow_vCPKRofP/left'
outfolder = '/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/siamese_eval_datasets/rotations90'
mkdirs(outfolder)

files = os.listdir(infolder)
for f in files:
    if f[0] == '.':
        continue
    im_orig = fm.load_pickle(os.path.join(infolder, f))
    fm.save_pickle(im_orig, os.path.join(outfolder, 'left', f))
    im_rot90 = ndi.rotate(im_orig, np.random.RandomState().choice([90,180,270]), axes=(1,2), reshape=False, order=5)
    fm.save_pickle(im_rot90, os.path.join(outfolder, 'right', f))

In [10]:
ev.visualize_simfolder(outfolder, figsize=5)

interactive(children=(Dropdown(description='Color', index=9, options=('inferno', 'viridis', 'prism', 'Greens',…